# Building X-Ray Classification Convolutional Neural Network Model

<img src="https://static.dw.com/image/16373135_401.jpg" alt="dw">

## Content

The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia/Normal). There are 5,863 X-Ray images (JPEG) and 2 categories (Pneumonia/Normal).

Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients of one to five years old from Guangzhou Women and Children’s Medical Center, Guangzhou. All chest X-ray imaging was performed as part of patients’ routine clinical care.

For the analysis of chest x-ray images, all chest radiographs were initially screened for quality control by removing all low quality or unreadable scans. The diagnoses for the images were then graded by two expert physicians before being cleared for training the AI system. In order to account for any grading errors, the evaluation set was also checked by a third expert.

## Acknowledgements

Data: https://data.mendeley.com/datasets/rscbjbr9sj/2

License: CC BY 4.0

Citation: http://www.cell.com/cell/fulltext/S0092-8674(18)30154-5

## Dataset and Sources

Picture: https://static.dw.com/image/16373135_401.jpg

Dataset: https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

## Importing Libraries

In [1]:
import time
import numpy as np
import pandas as pd
from keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

In [2]:
#classifier = load_model('model.h5')
#classifier.summary()
start = time.time()

## Data Preprocessing

### Import image data generator

In [3]:
from keras.preprocessing.image import ImageDataGenerator

ImadeDataGenerator library for pictures. The difference from normal picture readings is that it evaluates the pictures one by one, not all at once and helps the RAM to work in a healthy way.

In [4]:
## shear_range = Side bends
## zoom_range = Zoom
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

## target_size= 64x64 size pictures for scan.
## class_mode= Binary set
training_set = train_datagen.flow_from_directory("chest_xray/train",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory("chest_xray/test",
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Building Convolutional Neural Network Model

In [5]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPool2D

In [6]:
classifier = Sequential()
classifier.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
classifier.add(MaxPool2D(pool_size=2, strides=2))
classifier.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
classifier.add(MaxPool2D(pool_size=2, strides=2))
classifier.add(Flatten())
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid'))

### Train Convolutional Neural Network

In [8]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [9]:
classifier.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
163/163 [==============================] - 67s 409ms/step - loss: 0.3851 - accuracy: 0.8342 - val_loss: 0.2885 - val_accuracy: 0.8798
Epoch 2/25
163/163 [==============================] - 60s 369ms/step - loss: 0.2219 - accuracy: 0.9070 - val_loss: 0.3036 - val_accuracy: 0.8718
Epoch 3/25
163/163 [==============================] - 59s 363ms/step - loss: 0.2248 - accuracy: 0.9053 - val_loss: 0.2881 - val_accuracy: 0.8734
Epoch 4/25
163/163 [==============================] - 58s 355ms/step - loss: 0.2061 - accuracy: 0.9189 - val_loss: 0.2661 - val_accuracy: 0.9006
Epoch 5/25
163/163 [==============================] - 60s 369ms/step - loss: 0.1851 - accuracy: 0.9241 - val_loss: 0.4840 - val_accuracy: 0.8301
Epoch 6/25
163/163 [==============================] - 62s 383ms/step - loss: 0.1727 - accuracy: 0.9304 - val_loss: 0.4411 - val_accuracy: 0.8462
Epoch 7/25
163/163 [==============================] - 62s 381ms/step - loss: 0.1630 - accuracy: 0.9344 - val_loss: 0.3260 - val_ac

### Save the model

In [13]:
classifier.save('model.h5')
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

## Prediction

In [14]:
test_set.reset()
pred=classifier.predict_generator(test_set, verbose=1)

## Filter predictions
pred[pred > .5] = 1
pred[pred <= .5] = 0
print('Prediction successful.')

20/20 [==============================] - 5s 235ms/step
Prediction successful.


### Creating confusion matrix 

In [15]:
from sklearn.metrics import confusion_matrix

In [17]:
test_labels = []
for i in range(0, int(20)):
    test_labels.extend(np.array(test_set[i][1]))
print('Test Labels(test_labels):\n')
print(test_labels)

Test Labels(test_labels):

[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.

How each file was estimated and compared with real data is shown:

In [18]:
dosyaisimleri = test_set.filenames   
sonuc = pd.DataFrame()
sonuc['dosyaisimleri'] = dosyaisimleri
sonuc['tahminler'] = pred
sonuc['test'] = test_labels

In [19]:
cm = confusion_matrix(test_labels, pred)
print ("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[ 74 160]
 [123 267]]


### Prediction from Valitadion Pictures

In [21]:
from keras.preprocessing import image

#### Predict NORMAL class

In [22]:
image = image
image_name=[1427, 1430, 1431, 1436, 1437, 1438, 1440, 1442]
print('\nPrediction of NORMAL class')
for i in image_name:
    i = str(i)
    path = 'chest_xray/val/NORMAL/NORMAL2-IM-' + i + '-0001' + '.jpeg'
    img = image.load_img(path, target_size=(64, 64))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) 
    images = np.vstack([x])
    classes_normal = classifier.predict(images, batch_size=1)
    print(classes_normal)


Prediction of NORMAL class
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]


#### Predict PNEUMONIA class

In [23]:
person=[1946, 1947, 1949, 1950, 1951, 1952, 1954]
bacteria=[4875, 4876, 4880, 4881, 4882, 4883, 4886]
print('\nPrediction of PNEUMONIA class')
for i in person:
    index=int(person.index(i))
    bac=bacteria[index]  
    
    bac = str(bac)
    i = str(i)    
    path = 'chest_xray/val/PNEUMONIA/person' + i + '_bacteria_'+ bac +'.jpeg'
    img = image.load_img(path, target_size=(64, 64))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) 
    images = np.vstack([x])
    classes_pneumonia = classifier.predict(images, batch_size=1)
    print(classes_pneumonia)


Prediction of PNEUMONIA class
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]


In [24]:
end = time.time()
cal_time = end - start
print("\nTook {} seconds to classificate objects.".format(cal_time))


Took 1857.7696058750153 seconds to classificate objects.
